In [ ]:
import os
import sys
import logging
import traceback
import torch
import pickle
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.nn import GATv2Conv
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from time import time
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import optuna
from helper_functions import *
import seaborn as sns
import matplotlib.pyplot as plt


MAIN_PATH = "/dtu/3d-imaging-center/courses/02509/groups/group10/msc-hpc-run/"

# ---------------------------
# Global settings and seeding
# ---------------------------
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
pl.seed_everything(42)

# Global paths and constants
IMPORTS_PATH = os.path.join(MAIN_PATH, "data/import-volume.csv")
ADJACENCY_MATRIX = pd.read_parquet(os.path.join(MAIN_PATH, "data/adjacency-matrix.parquet"))
BOOKINGS_PATH = os.path.join(MAIN_PATH, "data/bookings_data.pkl")

# Global hyperparameters and settings
use_validation = False  
NODE_FEATURES = 21        # 6 from volume/week encoding + 13 booking features
TIME_WINDOW_SIZE = 13
LOADERS_WOKRES = 4

EARLY_STOP_PATIENCE = 7
EARLY_STOP_DELTA = 0.001

DIRECT_HORIZONS_TO_PREDICT = 13

# ---------------------------
# Data and Graph Preparation
# ----------------------
def get_import_data():
    """Load and pivot the import volume data."""
    try:
        import_data = pd.read_csv(IMPORTS_PATH)
        import_data["week"] = pd.to_datetime(import_data["week"])
        import_data["pool"] = import_data["pool"].astype("str")
        import_data["volume"] = import_data["import"].astype("float")
        import_data.drop(columns=["import"], inplace=True)
        import_data = import_data.sort_values("week")
        import_data = import_data.loc[
            (import_data.week >= pd.to_datetime("2017-05-01")) &
            (import_data.week <= pd.to_datetime("2024-10-20"))
        ]
        logging.info(f"Imports data range from {import_data['week'].min().strftime('%Y-%m-%d')} "
                     f"to {import_data['week'].max().strftime('%Y-%m-%d')}")
        import_data = import_data.pivot(index='week', columns='pool', values='volume').T
        import_data = import_data.fillna(0)
        return import_data
    except Exception as e:
        logging.error("Error in get_import_data(): " + str(e))
        raise

def get_graph_structure(threshold, a):
    """Construct the graph structure (edge_index and edge_weights) from the adjacency matrix."""
    try:
        a = a.reset_index(drop=True)
        a.columns = range(a.shape[1])
        a = a.where(pd.notnull(a), a.T)
        a = a.to_numpy()
        a_filtered = (a > threshold).astype(np.int32)
        edge_index = torch.nonzero(torch.tensor(a_filtered, dtype=torch.long), as_tuple=False).t()
        edge_weights = []
        for e in edge_index.numpy().T:
            distance = a[e[0], e[1]]
            edge_weights.append(distance)
        edge_weights = torch.tensor(edge_weights, dtype=torch.float32)
        return edge_index, edge_weights
    except Exception as e:
        logging.error("Error in get_graph_structure(): " + str(e))
        raise

def prepare_data(use_validation, prediction_horizon, batch_size):
    """Preprocess the data, create features, and return DataLoaders."""
    try:
        data = get_import_data()
        bookings = pickle.load(open(BOOKINGS_PATH, "rb"))
        logging.info(f"Bookings data range from {min(bookings.keys())} to {max(bookings.keys())}")

        if use_validation:
            val_weeks = 52
            test_weeks = 77
            train_weeks = data.shape[1] - val_weeks - test_weeks
        else:
            test_weeks = 77
            train_weeks = data.shape[1] - test_weeks

        volume_tensor = torch.tensor(data.values, dtype=torch.float32)
        week_numbers = data.columns.to_series().dt.isocalendar().week.astype(np.float32).values

        if use_validation:
            train_volume = volume_tensor[:, :train_weeks]
            val_volume = volume_tensor[:, train_weeks: train_weeks + val_weeks]
            test_volume = volume_tensor[:, train_weeks + val_weeks: train_weeks + val_weeks + test_weeks]
        else:
            train_volume = volume_tensor[:, :train_weeks]
            test_volume = volume_tensor[:, train_weeks: train_weeks + test_weeks]

        # Normalize volume using training statistics
        train_mean = train_volume.mean(dim=1, keepdim=True)
        train_std = train_volume.std(dim=1, keepdim=True) + 1e-6
        train_volume_norm = (train_volume - train_mean) / train_std
        if use_validation:
            val_volume_norm = (val_volume - train_mean) / train_std
        test_volume_norm = (test_volume - train_mean) / train_std

        # Create week encoding features
        week_numbers_tensor = torch.tensor(week_numbers, dtype=torch.float32)
        sin_week = torch.sin(2 * np.pi * week_numbers_tensor / 52)
        cos_week = torch.cos(2 * np.pi * week_numbers_tensor / 52)
        holiday_ohe = (week_numbers_tensor == 52).long()
        weeks_to_holiday = (52 - week_numbers_tensor) % 52
        weeks_from_holiday = (week_numbers_tensor - 52) % 52

        def split_features(feature, n_train, n_val, n_test):
            train_feat = feature[:n_train]
            val_feat = feature[n_train: n_train + n_val]
            test_feat = feature[n_train + n_val: n_train + n_val + n_test]
            return train_feat, val_feat, test_feat

        if use_validation:
            train_sin, val_sin, test_sin = split_features(sin_week, train_weeks, val_weeks, test_weeks)
            train_cos, val_cos, test_cos = split_features(cos_week, train_weeks, val_weeks, test_weeks)
            train_holiday, val_holiday, test_holiday = split_features(holiday_ohe, train_weeks, val_weeks, test_weeks)
            train_to, val_to, test_to = split_features(weeks_to_holiday, train_weeks, val_weeks, test_weeks)
            train_from, val_from, test_from = split_features(weeks_from_holiday, train_weeks, val_weeks, test_weeks)
        else:
            train_sin = sin_week[:train_weeks]
            test_sin = sin_week[train_weeks: train_weeks + test_weeks]
            train_cos = cos_week[:train_weeks]
            test_cos = cos_week[train_weeks: train_weeks + test_weeks]
            train_holiday = holiday_ohe[:train_weeks]
            test_holiday = holiday_ohe[train_weeks: train_weeks + test_weeks]
            train_to = weeks_to_holiday[:train_weeks]
            test_to = weeks_to_holiday[train_weeks: train_weeks + test_weeks]
            train_from = weeks_from_holiday[:train_weeks]
            test_from = weeks_from_holiday[train_weeks: train_weeks + test_weeks]

        def create_week_feature_tensor(week_feat, num_nodes):
            if week_feat.dim() == 1:
                week_feat = week_feat.unsqueeze(1)
            return week_feat.unsqueeze(0).repeat(num_nodes, 1, 1)

        num_nodes = 16  # Hard-coded based on the data
        train_sin_feat = create_week_feature_tensor(train_sin, num_nodes)
        train_cos_feat = create_week_feature_tensor(train_cos, num_nodes)
        train_holiday_feat = create_week_feature_tensor(train_holiday, num_nodes)
        train_to_feat = create_week_feature_tensor(train_to, num_nodes)
        train_from_feat = create_week_feature_tensor(train_from, num_nodes)

        if use_validation:
            val_sin_feat = create_week_feature_tensor(val_sin, num_nodes)
            val_cos_feat = create_week_feature_tensor(val_cos, num_nodes)
            val_holiday_feat = create_week_feature_tensor(val_holiday, num_nodes)
            val_to_feat = create_week_feature_tensor(val_to, num_nodes)
            val_from_feat = create_week_feature_tensor(val_from, num_nodes)
            test_sin_feat = create_week_feature_tensor(test_sin, num_nodes)
            test_cos_feat = create_week_feature_tensor(test_cos, num_nodes)
            test_holiday_feat = create_week_feature_tensor(test_holiday, num_nodes)
            test_to_feat = create_week_feature_tensor(test_to, num_nodes)
            test_from_feat = create_week_feature_tensor(test_from, num_nodes)
        else:
            test_sin_feat = create_week_feature_tensor(test_sin, num_nodes)
            test_cos_feat = create_week_feature_tensor(test_cos, num_nodes)
            test_holiday_feat = create_week_feature_tensor(test_holiday, num_nodes)
            test_to_feat = create_week_feature_tensor(test_to, num_nodes)
            test_from_feat = create_week_feature_tensor(test_from, num_nodes)

        train_volume_feat = train_volume_norm.unsqueeze(2)
        if use_validation:
            val_volume_feat = val_volume_norm.unsqueeze(2)
        test_volume_feat = test_volume_norm.unsqueeze(2)

        if use_validation:
            train_data_combined = torch.cat([
                train_volume_feat, train_sin_feat, train_cos_feat,
                train_holiday_feat, train_to_feat, train_from_feat
            ], dim=2)
            val_data_combined = torch.cat([
                val_volume_feat, val_sin_feat, val_cos_feat,
                val_holiday_feat, val_to_feat, val_from_feat
            ], dim=2)
            test_data_combined = torch.cat([
                test_volume_feat, test_sin_feat, test_cos_feat,
                test_holiday_feat, test_to_feat, test_from_feat
            ], dim=2)
        else:
            train_data_combined = torch.cat([
                train_volume_feat, train_sin_feat, train_cos_feat,
                train_holiday_feat, train_to_feat, train_from_feat
            ], dim=2)
            test_data_combined = torch.cat([
                test_volume_feat, test_sin_feat, test_cos_feat,
                test_holiday_feat, test_to_feat, test_from_feat
            ], dim=2)

        # Process booking features
        booking_list = []
        for date in data.columns:
            booking_df = bookings[date.strftime("%Y-%m-%d")]
            booking_list.append(booking_df.values)
        booking_array = np.stack(booking_list, axis=1)
        bookings_tensor = torch.tensor(booking_array, dtype=torch.float32)

        if use_validation:
            train_bookings = bookings_tensor[:, :train_weeks, :]
            val_bookings = bookings_tensor[:, train_weeks: train_weeks + val_weeks, :]
            test_bookings = bookings_tensor[:, train_weeks + val_weeks: train_weeks + val_weeks + test_weeks, :]
        else:
            train_bookings = bookings_tensor[:, :train_weeks, :]
            test_bookings = bookings_tensor[:, train_weeks: train_weeks + test_weeks, :]

        booking_mean = train_bookings.mean(dim=(0, 1), keepdim=True)
        booking_std = train_bookings.std(dim=(0, 1), keepdim=True) + 1e-6
        train_bookings = (train_bookings - booking_mean) / booking_std
        if use_validation:
            val_bookings = (val_bookings - booking_mean) / booking_std
        test_bookings = (test_bookings - booking_mean) / booking_std

        if use_validation:
            train_data_combined = torch.cat([train_data_combined, train_bookings], dim=2)
            val_data_combined = torch.cat([val_data_combined, val_bookings], dim=2)
            test_data_combined = torch.cat([test_data_combined, test_bookings], dim=2)
        else:
            train_data_combined = torch.cat([train_data_combined, train_bookings], dim=2)
            test_data_combined = torch.cat([test_data_combined, test_bookings], dim=2)

        class TimeSeriesDataset(Dataset):
            def __init__(self, data, window_size, horizon):
                self.data = data
                self.window_size = window_size
                self.horizon = horizon  # Now used to predict multiple future steps
                self.num_samples = data.shape[1] - window_size - horizon + 1

            def __len__(self):
                return self.num_samples

            def __getitem__(self, idx):
                x = self.data[:, idx: idx + self.window_size, :].transpose(0, 1)
                # Now y is a sequence of length `horizon` (i.e., 13 time steps)
                y = self.data[:, idx + self.window_size: idx + self.window_size + self.horizon, 0]
                return x, y

        window_size = TIME_WINDOW_SIZE
        train_dataset = TimeSeriesDataset(train_data_combined, window_size, horizon=prediction_horizon)
        test_dataset = TimeSeriesDataset(test_data_combined, window_size, horizon=prediction_horizon)
        if use_validation:
            val_dataset = TimeSeriesDataset(val_data_combined, window_size, horizon=prediction_horizon)

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, 
                                  pin_memory=True, num_workers=LOADERS_WOKRES)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True)
        if use_validation:
            val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, 
                                    pin_memory=True)
        else:
            val_loader = None

        return train_loader, val_loader, test_loader, (train_mean, train_std)
    except Exception as e:
        logging.error("Error in prepare_data(): " + str(e))
        traceback.print_exc()
        raise

def prepare_graph(threshold):
    """Prepare the graph structure (edge_index, edge_weights) using the given threshold."""
    try:
        edge_index, edge_weights = get_graph_structure(threshold, ADJACENCY_MATRIX)
        return edge_index, edge_weights
    except Exception as e:
        logging.error("Error in prepare_graph(): " + str(e))
        raise

# ---------------------------
# Model Definitions
# ---------------------------
class GNNLSTM(nn.Module):
    def __init__(self, in_channels, gnn_hidden, gat_heads, gat_dropout, gnn_dropout,
                 lstm_hidden, lstm_layers, lstm_dropout, forecast_horizon):
        super(GNNLSTM, self).__init__()
        self.gnn1 = GATv2Conv(
            in_channels=in_channels,
            out_channels=gnn_hidden,
            heads=gat_heads,
            concat=True,
            dropout=gat_dropout,
        )
        self.gnn2 = GATv2Conv(
            in_channels=gnn_hidden * gat_heads,
            out_channels=gnn_hidden,
            heads=1,
            concat=False,
        )
        self.lstm = nn.LSTM(
            input_size=gnn_hidden,
            hidden_size=lstm_hidden,
            batch_first=True,
            num_layers=lstm_layers,
            dropout=lstm_dropout,
        )
        # Now output a vector of length equal to forecast_horizon (i.e., 13 steps)
        self.fc = nn.Linear(lstm_hidden, forecast_horizon)
        
        self.norm1 = nn.LayerNorm(gnn_hidden * gat_heads)
        self.norm2 = nn.LayerNorm(gnn_hidden)
        self.relu = nn.ReLU()
        self.gnn_dropout = gnn_dropout

    def forward(self, x, edge_index, return_attention_weights=False):
        # x: (batch_size, seq_len, num_nodes, in_channels)
        batch_size, seq_len, num_nodes, _ = x.shape

        gnn_outputs = []
        for t in range(seq_len):
            gnn_out_batch = []
            for b in range(batch_size):
                node_features = x[b, t]
                if return_attention_weights:
                    gnn_out, attention_tuple = self.gnn1(node_features, edge_index, return_attention_weights=return_attention_weights)
                else:
                    gnn_out = self.gnn1(node_features, edge_index)
                gnn_out = self.norm1(gnn_out)
                # gnn_out = self.relu(gnn_out)
                # gnn_out = F.dropout(gnn_out, p=self.gnn_dropout, training=self.training)
                # gnn_out = self.gnn2(gnn_out, edge_index)
                # gnn_out = self.norm2(gnn_out)
                gnn_out_batch.append(gnn_out.unsqueeze(0))
            gnn_out_batch = torch.cat(gnn_out_batch, dim=0)
            gnn_outputs.append(gnn_out_batch.unsqueeze(1))
        gnn_out = torch.cat(gnn_outputs, dim=1)

        lstm_input = gnn_out.transpose(1, 2).reshape(batch_size * num_nodes, seq_len, -1)
        lstm_out, _ = self.lstm(lstm_input)
        lstm_out = lstm_out[:, -1, :]
        pred = self.fc(lstm_out)
        # Reshape to (batch_size, num_nodes, forecast_horizon)
        pred = pred.reshape(batch_size, num_nodes, -1)

        if return_attention_weights:
            return pred, attention_tuple
        else:
            return pred

class LitGNNLSTM(pl.LightningModule):
    def __init__(self, in_channels, gnn_hidden, gat_heads, gat_dropout, gnn_dropout,
                 lstm_hidden, lstm_layers, lstm_dropout, learning_rate, edge_index, forecast_horizon):
        super(LitGNNLSTM, self).__init__()
        self.model = GNNLSTM(in_channels, gnn_hidden, gat_heads, gat_dropout, gnn_dropout,
                             lstm_hidden, lstm_layers, lstm_dropout, forecast_horizon)
        self.learning_rate = learning_rate
        self.criterion = nn.MSELoss()
        self.register_buffer("edge_index", edge_index)

    def forward(self, x, return_attention_weights=False):
        return self.model(x, self.edge_index, return_attention_weights)

    def training_step(self, batch, batch_idx):
        x, y = batch  # y now has shape (batch_size, num_nodes, forecast_horizon)
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, on_step=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, on_step=False)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("test_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.learning_rate)
        # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1, verbose=True)
        return optimizer
         

def create_model(edge_index, params):
    """Create the Lightning model using hyperparameters from Optuna."""
    try:
        model = LitGNNLSTM(
            in_channels=NODE_FEATURES,
            gnn_hidden=params["gnn_hidden"],
            gat_heads=params["gat_heads"],
            gat_dropout=params["gat_dropout"],
            gnn_dropout=params["gnn_dropout"],
            lstm_hidden=params["lstm_hidden"],
            lstm_layers=params["lstm_layers"],
            lstm_dropout=params["lstm_dropout"],
            learning_rate=params["learning_rate"],
            edge_index=edge_index,
            forecast_horizon=DIRECT_HORIZONS_TO_PREDICT,  # Now predicting 13 steps at once
        )
        return model
    except Exception as e:
        logging.error("Error in create_model(): " + str(e))
        raise


# ---------------------------
# Trainer Setup
# ---------------------------
def create_trainer(max_epochs):
    """Create a PyTorch Lightning Trainer with early stopping and AMP enabled."""
    early_stop_callback = EarlyStopping(
        monitor="val_loss",
        min_delta=EARLY_STOP_DELTA,
        patience=EARLY_STOP_PATIENCE,
        verbose=True,
        mode="min",
    )
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="gpu",
        devices=1,
        precision="16-mixed",
        callbacks=[early_stop_callback],
        log_every_n_steps=1,
        enable_progress_bar=False,
        enable_checkpointing=True,
        logger=True,
        enable_model_summary=False,
        default_root_dir="lightning_gat_weights/"
    )
    return trainer

Seed set to 42


In [2]:
PARAMS_PATH = MAIN_PATH + f"output_old/sho/study_results_gat/sho_gat_trials_df.parquet"

In [3]:
params_df = pd.read_parquet(PARAMS_PATH).sort_values("value")
best_params = params_df.head(1)

In [4]:
best_params

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_gat_dropout,params_gat_heads,params_gnn_dropout,params_gnn_hidden,params_graph_threshold,params_learning_rate,params_lstm_dropout,params_lstm_hidden,params_lstm_layers,user_attrs_epochs,state
188,188,0.762765,2025-03-18 11:24:05.114911,2025-03-18 11:24:50.350509,0 days 00:00:45.235598,24,0.3,9,0.2,880,20,0.000024,0.6,656,2,70,COMPLETE


In [5]:
# params = {
#     "learning_rate": float(best_params["params_learning_rate"].values[0]),
#     "gnn_hidden": int(best_params["params_gnn_hidden"].values[0]),
#     "gnn_dropout": float(best_params["params_gnn_dropout"].values[0]),
#     "gat_heads": int(best_params["params_gat_heads"].values[0]),
#     "gat_dropout": float(best_params["params_gat_dropout"].values[0]),
#     "lstm_hidden": int(best_params["params_lstm_hidden"].values[0]),
#     "lstm_dropout": float(best_params["params_lstm_dropout"].values[0]),
#     "lstm_layers": int(best_params["params_lstm_layers"].values[0]),
#     "graph_threshold": int(best_params["params_graph_threshold"].values[0]),
#     "batch_size": int(best_params["params_batch_size"].values[0]),
#     "epochs": 300,
# }

In [6]:
params = {
    "learning_rate": 0.001,
    "gnn_hidden": 512,
    "gnn_dropout": 0.4,
    "gat_heads": 8,
    "gat_dropout": 0.6,
    "lstm_hidden": 512,
    "lstm_dropout": 0.6,
    "lstm_layers": 2,
    "graph_threshold": 0,
    "batch_size": 16,
    "epochs": 300,
}

In [7]:
train_loader, val_loader , test_loader, train_statistics = prepare_data(use_validation=use_validation, prediction_horizon=DIRECT_HORIZONS_TO_PREDICT, batch_size=params["batch_size"])
train_mean, train_std = train_statistics
edge_index, _ = prepare_graph(params["graph_threshold"])

model = create_model(edge_index, params)
trainer = create_trainer(max_epochs=params["epochs"])
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=test_loader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2025-03-20 14:09:00.478831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742476140.499827  322727 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742476140.506335  322727 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742476140.523058  322727 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742476140.523081  322727 computation_placer.cc:177] computation placer already registered

OSError: Writer is closed

In [ ]:
model.eval()
all_preds = []
all_targets = []
attention_tuples = []
with torch.no_grad():
    for x, y in test_loader:
        preds, attention_tuple = model(x, return_attention_weights=True)
        all_preds.append(preds.cpu().numpy())
        all_targets.append(y.numpy())
        attention_tuples.append(attention_tuple)

all_preds = np.concatenate(all_preds, axis=0)
all_targets = np.concatenate(all_targets, axis=0)

all_preds = (all_preds.T * train_std.numpy() + train_mean.numpy()).T
all_targets = (all_targets.T * train_std.numpy() + train_mean.numpy()).T

In [ ]:
att_coefs = []
for b in range(len(attention_tuples)):
    att_tuple = attention_tuples[b]
    att_w_idx = att_tuple[0].numpy()
    att_coef = att_tuple[1].numpy().mean(axis=1).reshape(-1, 1)
    att_coefs.append(att_coef)
att_coefs = np.hstack(att_coefs).mean(axis=1).reshape(-1, 1)

In [ ]:
data = get_import_data()

edges = att_w_idx
weights = att_coef.flatten()

matrix = np.zeros((16, 16))

for i in range(edges.shape[1]):
    source = int(edges[0, i])
    target = int(edges[1, i])
    matrix[source, target] = weights[i]

plt.figure(figsize=(10, 8))
ax = sns.heatmap(matrix, cmap="viridis", vmin=0, vmax=1,
                 xticklabels=data.index.str[2:].tolist(), yticklabels=data.index.str[2:].tolist(),
                 annot=True, fmt=".2f")

# Move the x-axis labels to the top.
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')

ax.set_xlabel("Target Node")
ax.set_ylabel("Source Node")
ax.set_title("Attention Weights", pad=30)  # Increase pad to give space for the title

plt.show()


In [ ]:
predictions = create_predictions(all_preds, all_targets)
results_horizon = create_results_horizon(predictions)
results_pool = create_results_pool(predictions)

In [ ]:
results_horizon

In [ ]:
results_pool

In [ ]:
visualize_errors(predictions)

In [ ]:
# Test RMSE: 205.0331
# Test MPE: -10.3120
# Test WMAPE: 20.0702

In [ ]:
visualize_results(predictions, "GBFXS")